In [ ]:
# Configures the Environment (run it once)
include("../src/configure.jl")

# Configuration of the Quadrotor

In [ ]:
include("../src/Model_nd_Dynamics.jl")
include("../src/Simulation.jl")
include("../src/MPCTunningParams.jl")
include("../src/Trajectory.jl")
include("../src/MPC.jl")
include("../src/Close_loop.jl")
include("../src/LQR.jl")

In [ ]:
model = Model(0.0, 0.0, 0.0, I(3), 0.0, 0.0, 0.0, zeros(4), zeros(4),0.0,0.0, 0, 0, I(3))
model.g = 9.81 # [m/s^2]
model.m = 0.028 # [kg]
model.ℓ = 0.0460 # [meters]
model.J = 1e-6 * [
    16.571710   0.830806   0.718277
     0.830806  16.655602   1.800197
     0.718277   1.800197  29.261652
] # [kg . m^2]
model.R = 0.0235 # [meters]
model.prop_min_h = 0.1 # [meters]
model.ρ = 3.6 # dimensionless
model.kt = 1.28e-8
model.km = model.kt * 0.00596
model.Nx = 13
model.Nu = 4

#Thrust limits
model.umin = 0 * model.g * ones(4)
model.umax = 0.58 * model.m * model.g * ones(4)

model.K_aero = 1e-7 * [ 9.1785   0.0000   0.0000;
                         0.0000   9.1785   0.0000;
                         0.0000   0.0000   10.311 ]

## Simulation Configs
#
# Universe 1000Hz
# Controller 100Hz

h_universe = 0.0001 # Time step (10 kHz)
h_controller = 0.002 # Time step (500 Hz)
Tfinal = 10.0 # Simulate from 0 - 10 seconds

simulation = Simulation(h_universe, h_controller, Tfinal)

landing_start_height = 3.0
stationary_height = model.prop_min_h + 0.06 # 6 centimeters above the ground
plat_amplitude = 0.1
plat_freq = 0.3

traj_params = Trajectory(plat_amplitude, plat_freq, landing_start_height, stationary_height)

state_space = LinearStateSpace(model, simulation, (model.Nx - 1), model.Nu, stationary_height)

Nh = 20
Nc = 19 # Vai de 0 até (Nh -1)

# Cost weights
#Q = Array(10.0*I(Nx));
#Q[3,3] = 1000.0
#R = Array(.01*I(Nu));
Q = Array(100.0*I(model.Nx - 1));
R = Array(.1*I(model.Nu));

tunning_params = MPCTunningParameters(Nh, Nc, Q, R)

mpc_mats =  MPCMatrices(model, state_space, tunning_params)

# Initial State!
theta = 45 * pi / 180  # 10 graus em rad
q0 = [cos(theta/2), sin(theta/2), 0.0, 0.0]
x0 = [1.0; 2.0; traj_params.init_heigth; q0 ; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0]

wind_vec = [0.0, 1.0, 0.0]
wind_velocity = 0.0

In [ ]:
out = closed_loop(
    x0,
    state_space,
    tunning_params, 
    traj_params,
    simulation,
    model,

    (t, x) -> mpc_controller(
        model,
        simulation,
        tunning_params,
        traj_params,
        state_space,
        mpc_mats,
        t,
        x, 
        gen_ref(model, traj_params, tunning_params, t, simulation.h_controller) # verify if this works as expected!!
    ),
    wind_vec,
    wind_velocity
);

In [ ]:
out_lqr = closed_loop(
    x0,
    state_space,
    tunning_params, 
    traj_params,
    simulation,
    model,

    (t, x) -> lqr_controller(
        model,
        simulation,
        tunning_params,
        traj_params,
        state_space,
        t,
        x, 
        gen_ref(model, traj_params, tunning_params, t, simulation.h_controller) # verify if this works as expected!!
    ),
    wind_vec,
    wind_velocity
);

In [ ]:
include("../src/Report.jl")

thist = Array(range(0, simulation.h_universe * (simulation.Nt_universe - 1), step = simulation.h_universe));

In [ ]:
plot_state(1, "x", "m", out, out_lqr, thist)

In [ ]:
plot_state(2, "y", "m", out, out_lqr, thist)

In [ ]:
plot_state(3, "z", "m", out, out_lqr, thist)

In [ ]:
plot_state(8, "dx", "m/s", out, out_lqr, thist)

In [ ]:
plot_state(9, "dy", "m/s", out, out_lqr, thist)

In [ ]:
plot_state(10, "dz", "m/s", out, out_lqr, thist)

In [ ]:
plot_attitude_error(out, out_lqr, thist)

In [ ]:
plot_u(1, out, out_lqr, thist, model)

In [ ]:
plot_u(2, out, out_lqr, thist, model)

In [ ]:
plot_u(3, out, out_lqr, thist, model)

In [ ]:
plot_u(4, out, out_lqr, thist, model)

In [ ]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

In [ ]:
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

In [ ]:
X1 = [SVector{13}(x) for x in eachcol(out.x_quad[:,1:10:end])];

visualize!(vis, quad_model, thist[end], X1)